---

_You are currently looking at **version** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    towns = []
    with open('university_towns.txt') as file:
        for line in file.readlines():
            if not re.match(r'^\s*$', line):
                towns.append(line)
                
    for index, line in enumerate(towns):
        towns[index] = line.strip()
        
    df = pd.DataFrame(columns=('State', 'RegionName'))
    i = 0
    state = ""
    region = ""
    for line in towns:
        if '[edit]' in line:
            state = line.replace('[edit]', "")
        else:
            region = re.sub(r' \(.*', "", line)
            df.loc[i] = [state, region]
            i += 1
    
    return df

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    gdp = pd.ExcelFile('gdplev.xls').parse(skiprows=7)[['Unnamed: 4', 'Unnamed: 5']].loc[212:]
    gdp.columns = ['Quarter', 'GDP']
    gdp['GDP'] = gdp['GDP'].astype('float32')

    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            return gdp.iloc[i][0]

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    gdp = pd.ExcelFile('gdplev.xls').parse(skiprows=7)[['Unnamed: 4', 'Unnamed: 5']].loc[212:]
    gdp.columns = ['Quarter', 'GDP']
    gdp['GDP'] = gdp['GDP'].astype('float32')
    
    start = 0
    
    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            start = i
            for j in range (start, len(gdp) - 2):
                if (gdp.iloc[j][1] < gdp.iloc[j+1][1]) & (gdp.iloc[j+1][1] < gdp.iloc[j+2][1]):
                    return gdp.iloc[j+2][0]

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.ExcelFile('gdplev.xls').parse(skiprows=7)[['Unnamed: 4', 'Unnamed: 5']].loc[212:]
    gdp.columns = ['Quarter', 'GDP']
    gdp['GDP'] = gdp['GDP'].astype('float32')
    
    start = 0
    end = 0
    
    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            start = i
            for j in range (start, len(gdp) - 2):
                if (gdp.iloc[j][1] < gdp.iloc[j+1][1]) & (gdp.iloc[j+1][1] < gdp.iloc[j+2][1]):
                    end = j
                    
    recession = gdp.iloc[start:end]
    return recession['Quarter'].loc[recession['GDP'].idxmin()]

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

    df = pd.read_csv('City_Zhvi_AllHomes.csv', header=0)
    
    columns = ['RegionID', 'RegionName', 'State']
    for i in range(2000, 2017):
        for j in range(1, 13):
            if j <=9:
                if i == 2016 and j == 9:
                    pass
                else:
                    month = '0' + str(j)
            else:
                if i == 2016:
                    pass
                else:
                    month = str(j)
    
            columns.append(str(i) + '-' + month)
    
    df = df[columns]
    df['State'] = df['State'].replace(states)
    
    def convert_quarter(date):
        year, month = date.split('-')
        if month in ('01', '02', '03'):
            output = year + 'q1'
        elif month in ('04', '05', '06'):
            output = year + 'q2'
        elif month in ('07', '08', '09'):
            output = year + 'q3'
        else:
            output = year + 'q4'
        return output
    
    df_2 = df.copy().set_index(['State', 'RegionName', 'RegionID']).stack(dropna=False)
    df_2 = df_2.reset_index().rename(columns={'level_3': 'Period', 0: 'GDP'})
    df_2.drop_duplicates(inplace=True)
    df_2['Quarter'] = df_2['Period'].apply(convert_quarter)
    
    return df_2.pivot_table(values='GDP', index=['State', 'RegionName', 'RegionID'], columns='Quarter', aggfunc=np.mean).reset_index().drop('RegionID', axis=1).set_index(['State', 'RegionName'])

In [10]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    uni_towns = get_list_of_university_towns()
    uni_towns['UT'] = True
    
    df = pd.merge(convert_housing_data_to_quarters(), uni_towns, how='left', left_index=True, right_on=['State', 'RegionName'])
    df['UT'] = df['UT'].replace({np.NaN: False})
    
    start = get_recession_start()
    bottom = get_recession_bottom()
    
    start_y = int(start.split('q')[0])
    start_q = int(start.split('q')[1])
    bottom_y = int(bottom.split('q')[0])
    bottom_q = int(bottom.split('q')[1])
    
    before = str(start_y) + 'q' + str(start_q-1)
    
    columns = ['State', 'RegionName', 'UT']
    quarters = []
    for i in range(start_y, bottom_y+1):
        for j in range(1, 5):
            if (i == start_y and j < start_q) or (i == bottom_y and j > bottom_q):
                pass
            else:
                quarters.append(str(i) + 'q' + str(j))
                
    quarters.append(before)
    
    df = df[columns + quarters]
    
    df['Ratio'] = df[before] / df[bottom]
    
    UT = df[df['UT']==True]['Ratio']
    nonUT = df[df['UT']==False]['Ratio']
    stat, p = ttest_ind(UT, nonUT, nan_policy='omit')
    
    different = False
    if p < 0.01:
        different = True
    
    better = ""
    if UT.mean() < nonUT.mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, p, better)